# Create Dataset

* [tf.data: Build TensorFlow input pipelines](https://www.tensorflow.org/guide/data) (MUST)
* [Load CSV data](https://www.tensorflow.org/tutorials/load_data/csv)
* [tf.data.experimental.make_csv_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset)


In [2]:
import tensorflow as tf
import pandas as pd

# Example CSV

Titanic CSV.

In [3]:
titanic_file = tf.keras.utils.get_file("titanic_train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")

In [4]:
!head -n 5  ~/.keras/datasets/titanic_train.csv

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n


# Dictionary to Dataset

In [18]:
features = {
    "name": ["hoge", "tako"],
    "age": [20, 99]
}
labels = {
    "survived": [0, 1]
}
ds_from_dictionary = tf.data.Dataset.from_tensor_slices((features, labels))
for record in ds_from_dictionary:
    print(record)

({'name': <tf.Tensor: shape=(), dtype=string, numpy=b'hoge'>, 'age': <tf.Tensor: shape=(), dtype=int32, numpy=20>}, {'survived': <tf.Tensor: shape=(), dtype=int32, numpy=0>})
({'name': <tf.Tensor: shape=(), dtype=string, numpy=b'tako'>, 'age': <tf.Tensor: shape=(), dtype=int32, numpy=99>}, {'survived': <tf.Tensor: shape=(), dtype=int32, numpy=1>})


# Pandas to Dataset

If  data fits in memory, ```from_tensor_slices``` method works on dictionaries.

```Pandas -> Python dictionary -> from_tensor_slices -> Dataset```

In [5]:
df = pd.read_csv(titanic_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   survived            627 non-null    int64  
 1   sex                 627 non-null    object 
 2   age                 627 non-null    float64
 3   n_siblings_spouses  627 non-null    int64  
 4   parch               627 non-null    int64  
 5   fare                627 non-null    float64
 6   class               627 non-null    object 
 7   deck                627 non-null    object 
 8   embark_town         627 non-null    object 
 9   alone               627 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 49.1+ KB


In [33]:
type(df.loc[:, ['survived']])

pandas.core.frame.DataFrame

In [44]:
titanic_from_pandas = tf.data.Dataset.from_tensor_slices((
    dict(df.loc[:, df.columns != 'survived']),
    dict(df.loc[:, ['survived']])
))
for row in titanic_from_pandas.batch(1).take(1):  # Take the first batch 
    features = row[0]        # Diectionary
    label = row[1]
    
    for feature, value in features.items():
        print(f"{feature:20s}: {value}")
    
    print(f"label/survived      : {label}")    

sex                 : [b'male']
age                 : [22.]
n_siblings_spouses  : [1]
parch               : [0]
fare                : [7.25]
class               : [b'Third']
deck                : [b'unknown']
embark_town         : [b'Southampton']
alone               : [b'n']
label/survived      : {'survived': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>}


In [6]:
titanic_from_pandas = tf.data.Dataset.from_tensor_slices(dict(df))
titanic_from_pandas.take(1)

2021-10-30 18:16:37.932451: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<TakeDataset shapes: {survived: (), sex: (), age: (), n_siblings_spouses: (), parch: (), fare: (), class: (), deck: (), embark_town: (), alone: ()}, types: {survived: tf.int64, sex: tf.string, age: tf.float64, n_siblings_spouses: tf.int64, parch: tf.int64, fare: tf.float64, class: tf.string, deck: tf.string, embark_town: tf.string, alone: tf.string}>

# Read CSV into Dataset

```make_csv_dataset``` method reads the csv and split them into ```(features, label)``` where ```features``` is a diectioay of ```(feature, value)```.

In [42]:
titanic = tf.data.experimental.make_csv_dataset(
    titanic_file,
    label_name="survived",
    batch_size=1,   # To compre with the head of CSV
    shuffle=False,  # To compre with the head of CSV
    header=True,
)

In [43]:
for row in titanic.take(1):  # Take the first batch 
    features = row[0]        # Diectionary
    label = row[1]
    
    for feature, value in features.items():
        print(f"{feature:20s}: {value}")
    
    print(f"label/survived      : {label}")    

sex                 : [b'male']
age                 : [22.]
n_siblings_spouses  : [1]
parch               : [0]
fare                : [7.25]
class               : [b'Third']
deck                : [b'unknown']
embark_town         : [b'Southampton']
alone               : [b'n']
label/survived      : [0]


2021-10-30 18:49:14.543589: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-30 18:49:14.545234: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 1996235000 Hz
